In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
DATA_DIR = "C:/Users/91701/Major demo1/H"
OUTPUT_DIR = "C:/Users/91701/Major demo1/outputs"
CHECKPOINT_DIR = os.path.join(OUTPUT_DIR, "checkpoints")
LOG_DIR = os.path.join(OUTPUT_DIR, "logs")

In [ ]:
IMAGE_SIZE = 128
BATCH_SIZE = 8
EPOCHS = 700
LATENT_DIM = 100
N_CRITIC = 5  # Discriminator updates per generator update
LAMBDA_GP = 10  # Gradient penalty weight
LEARNING_RATE = 5e-5
G_LEARNING_RATE = 1e-4

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

In [ ]:
device_name = "GPU" if tf.config.list_physical_devices('GPU') else "CPU"
print(f"Using device: {device_name}")

In [ ]:
def load_images():
    images = []
    for file in os.listdir(DATA_DIR):
        if file.lower().endswith(('png','jpg','jpeg')):
            img = Image.open(os.path.join(DATA_DIR, file)).convert('RGB').resize((IMAGE_SIZE, IMAGE_SIZE))
            img = np.array(img, dtype=np.float32)
            img = (img / 127.5) - 1.0  # Normalize to [-1, 1]
            images.append(img)
    return np.array(images)

images_np = load_images()
print(f"Loaded dataset: {images_np.shape}")

dataset = tf.data.Dataset.from_tensor_slices(images_np)

In [ ]:
def augment(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image

dataset = dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=len(images_np)).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
def build_generator(latent_dim):
    model = tf.keras.Sequential([
        layers.Input(shape=(latent_dim,)),
        layers.Dense(8*8*256, use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 256)),

        layers.Conv2DTranspose(128, 5, strides=2, padding="same", use_bias=False),
        layers.BatchNormalization(), layers.LeakyReLU(),

        layers.Conv2DTranspose(64, 5, strides=2, padding="same", use_bias=False),
        layers.BatchNormalization(), layers.LeakyReLU(),

        layers.Conv2DTranspose(32, 5, strides=2, padding="same", use_bias=False),
        layers.BatchNormalization(), layers.LeakyReLU(),

        layers.Conv2DTranspose(3, 5, strides=2, padding="same", activation='tanh')
    ])
    return model

In [ ]:
def build_discriminator(img_shape):
    model = tf.keras.Sequential([
        layers.Input(shape=img_shape),
        layers.Conv2D(64, 5, strides=2, padding="same"),
        layers.LeakyReLU(0.2),

        layers.Conv2D(128, 5, strides=2, padding="same"),
        layers.LayerNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(256, 5, strides=2, padding="same"),
        layers.LayerNormalization(),
        layers.LeakyReLU(0.2),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

generator = build_generator(LATENT_DIM)
discriminator = build_discriminator((IMAGE_SIZE, IMAGE_SIZE, 3))


In [ ]:
def gradient_penalty(batch_size, real_images, fake_images):
    alpha = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0)
    interpolated = alpha * real_images + (1 - alpha) * fake_images

    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        pred = discriminator(interpolated, training=True)
    grads = tape.gradient(pred, [interpolated])[0]
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    return gp

def generator_loss(fake_pred):
    return -tf.reduce_mean(fake_pred)

def discriminator_loss(real_pred, fake_pred):
    return tf.reduce_mean(fake_pred) - tf.reduce_mean(real_pred)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(G_LEARNING_RATE, beta_1=0.0, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.0, beta_2=0.9)

In [ ]:
checkpoint = tf.train.Checkpoint(generator=generator, discriminator=discriminator,
                                 g_optimizer=generator_optimizer, d_optimizer=discriminator_optimizer)
manager = tf.train.CheckpointManager(checkpoint, CHECKPOINT_DIR, max_to_keep=5)

summary_writer = tf.summary.create_file_writer(LOG_DIR)

In [ ]:
@tf.function
def train_step(real_images):
    batch_size = tf.shape(real_images)[0]

    for _ in range(N_CRITIC):
        noise = tf.random.normal([batch_size, LATENT_DIM])

        with tf.GradientTape() as disc_tape:
            fake_images = generator(noise, training=True)
            real_pred = discriminator(real_images, training=True)
            fake_pred = discriminator(fake_images, training=True)

            gp = gradient_penalty(batch_size, real_images, fake_images)
            d_loss = discriminator_loss(real_pred, fake_pred) + LAMBDA_GP * gp

        d_grad = disc_tape.gradient(d_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(d_grad, discriminator.trainable_variables))

    noise = tf.random.normal([batch_size, LATENT_DIM])
    with tf.GradientTape() as gen_tape:
        fake_images = generator(noise, training=True)
        fake_pred = discriminator(fake_images, training=True)
        g_loss = generator_loss(fake_pred)

    g_grad = gen_tape.gradient(g_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(g_grad, generator.trainable_variables))

    return g_loss, d_loss

In [ ]:
def generate_and_save_images(epoch):
    test_input = tf.random.normal([16, LATENT_DIM])
    predictions = generator(test_input, training=False)
    predictions = (predictions + 1.0) / 2.0

    grid_img = np.zeros((IMAGE_SIZE*4, IMAGE_SIZE*4, 3))
    for i in range(4):
        for j in range(4):
            img = predictions[i*4+j].numpy()
            grid_img[i*IMAGE_SIZE:(i+1)*IMAGE_SIZE, j*IMAGE_SIZE:(j+1)*IMAGE_SIZE, :] = img

    Image.fromarray((grid_img*255).astype(np.uint8)).save(os.path.join(OUTPUT_DIR, f"epoch_{epoch:03d}.png"))

In [ ]:
for epoch in range(1, EPOCHS+1):
    g_losses, d_losses = [], []
    for real_images in tqdm(dataset, desc=f"Epoch {epoch}/{EPOCHS}"):
        g_loss, d_loss = train_step(real_images)
        g_losses.append(g_loss.numpy())
        d_losses.append(d_loss.numpy())

    with summary_writer.as_default():
        tf.summary.scalar('Generator Loss', np.mean(g_losses), step=epoch)
        tf.summary.scalar('Discriminator Loss', np.mean(d_losses), step=epoch)

    if epoch % 5 == 0:
        generate_and_save_images(epoch)
        manager.save()

    print(f"Epoch {epoch} | Gen Loss: {np.mean(g_losses):.4f} | Disc Loss: {np.mean(d_losses):.4f}")